Importing Libraries

In [1]:
import re
import os
import s3fs
from data_ingestion import ingest_data_from_s3
from pyspark.sql import SparkSession

In [2]:
# s3_path = 'datasci-assignment/'
# #Create S3 object to read from public S3-bucket.
# s3 = s3fs.S3FileSystem(anon =  True)

# #Start INgesting data in Landing folder.
# ingest_data_from_s3(os.getcwd(), s3, s3_path)
print(str(os.getcwd()).replace("\\", "/"))

: 

In [2]:
print(str(os.getcwd()).replace("\\", "/"))

c:/Users/Admin/Downloads/read_public_s3


In [3]:

spark = SparkSession.builder.master("local[*]").appName("click_logs").getOrCreate()

In [96]:
from pyspark.sql.functions import *

df = spark.read.option("InferSchema", "True").option("mode", "PERMISSIVE").option("multiLine", True).json("C:/Users/Admin/Downloads/read_public_s3/Landing/click_log/2024/05/10/00/")
df = df.select('*', "ip_geo.*", "query.*").drop("query", "ip_geo")
# df.write.format("parquet")
df = df.withColumn("real_filepath", input_file_name())

df = df.withColumn("actual_file" , split(df.real_filepath, '/',limit=-1))
df = df.withColumn("count_file", size(df.actual_file))
df = df.withColumn("actual_file" , df.actual_file[col("count_file")-1]).drop("count_file")
# df.withColum("filepath", F.regexp_extract("filepath", "State=(.+)\.snappy\.parquet", 1)

In [97]:
df.show(truncate=False)

+-----------+-------------+-----------+------------------------------------+------------------------------------+------------------------------------+---------------+--------------+-------------------------------------------------------------------+------------+--------------------+----------------------------------------------+-------------+-------+----------+-------+-------+-------+----------+-----+------+-----------+-------------------------------------------------------------------+-----------+------------------------------------+------------------------------------+---------------+----------+----------+------------+----------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------

In [133]:
df = spark.read.schema(a).option("mode", "PERMISSIVE").option("multiLine", True).json("C:/Users/Admin/Downloads/read_public_s3/Landing/postback-log/2024/05/10/00/performance_postback_log-1-2024-05-10-00-00-33-efc64b50-edbc-4a15-97ae-0aaefa93ce79")
# df = df.select('*', "ip_geo.*", "query.*").drop("query", "ip_geo")
# # df.write.format("parquet")
# df = df.withColumn("real_filepath", input_file_name())

# df = df.withColumn("actual_file" , split(df.real_filepath, '/',limit=-1))
# df = df.withColumn("count_file", size(df.actual_file))
# df = df.withColumn("actual_file" , df.actual_file[col("count_file")-1]).drop("count_file")
# # df.withColum("filepath", F.regexp_extract("filepath", "State=(.+)\.snappy\.parquet", 1)

df.show()

PySparkTypeError: [NOT_STR_OR_STRUCT] Argument `schema` should be a str or structType, got LongType.

In [141]:
for i in df.columns:
    # print(i)
    a = df[i].dataType
    print(a)


#Get All column names and it's types
for field in df.schema.fields:
    # print(field)
    print(field.name +" , "+str(field.dataType))

Column<'click_config[dataType]'>
Column<'query[dataType]'>
Column<'req[dataType]'>
Column<'timestamp[dataType]'>
click_config , StructType([StructField('campaign_id', StringType(), True), StructField('click_id', StringType(), True), StructField('click_time', LongType(), True), StructField('creative_id', StringType(), True), StructField('device_id', StringType(), True), StructField('dummy_publisher_id', StringType(), True), StructField('events', StructType([StructField('0', StructType([StructField('ev', StringType(), True), StructField('tm', LongType(), True)]), True), StructField('1', StructType([StructField('ev', StringType(), True), StructField('pubEventId', LongType(), True), StructField('query', StructType([StructField('attribution_type', StringType(), True)]), True), StructField('sendToPub', LongType(), True), StructField('status', LongType(), True), StructField('ts', LongType(), True)]), True), StructField('2', StructType([StructField('ev', StringType(), True), StructField('pubEv

In [99]:
df.printSchema()

root
 |-- click_config: struct (nullable = true)
 |    |-- campaign_id: string (nullable = true)
 |    |-- click_id: string (nullable = true)
 |    |-- click_time: long (nullable = true)
 |    |-- creative_id: string (nullable = true)
 |    |-- device_id: string (nullable = true)
 |    |-- dummy_publisher_id: string (nullable = true)
 |    |-- events: struct (nullable = true)
 |    |    |-- 0: struct (nullable = true)
 |    |    |    |-- ev: string (nullable = true)
 |    |    |    |-- tm: long (nullable = true)
 |    |    |-- 1: struct (nullable = true)
 |    |    |    |-- ev: string (nullable = true)
 |    |    |    |-- pubEventId: long (nullable = true)
 |    |    |    |-- query: struct (nullable = true)
 |    |    |    |    |-- attribution_type: string (nullable = true)
 |    |    |    |-- sendToPub: long (nullable = true)
 |    |    |    |-- status: long (nullable = true)
 |    |    |    |-- ts: long (nullable = true)
 |    |    |-- 2: struct (nullable = true)
 |    |    |    |-- 

In [255]:
a = []
for i in df.dtypes:

    print(i[0])
    # print(len(i), isinstance(df.schema[i[0]].dataType, StructType))
    if isinstance(df.schema[i[0]].dataType, StructType):
        print(df.schema[i[0]].dataType.fieldNames())
        a += list(map(lambda column_name: f'{i[0]}.{column_name}' ,df.schema[i[0]].dataType.fieldNames()))
    else:
        a.append(i[0])
    # print(i[1])
    # a = i[1].replace("struct<", "").replace(">", "")
    # print(to_json(a))
    # ddl_schema = StructType.fromDDL(a)

    # Show the generated schema
    # ddl_schema.printSchema()
    # break
# print(list(list(column_structure)[0]))
print(a)

campaign_id
click_id
click_time
creative_id
device_id
dummy_publisher_id
events
['0', '1', '2', '3', '4']
gaid
gross_adv_cost
idfa
install_count
install_time
net_adv_cost
p_click_id
pub_payout
pub_valid_conversion
publisher_id
status
status_code
sub_campaign_id
sub_publisher_id
timestamp
va_total_conversion
va_valid_conversion
valid_click
adid
app_type
app_version
attribution_type
bundle_id
bundle_name
campaign_id
campaign_name
car
carrier_name
city
click_id
click_utc
country
creative_name
ctype
device_brand
device_id
device_model
device_type
device_version
event_id
event_name
event_timestamp
event_value
gaid
install_utc
lng
mmp
op
original_pid
os
os_version
publisher_id
re
ree
rt
sdk
session_ip
st
sub_id2
sub_publisher_id
transaction_amount
urlClickId
user_agent
wifi
req
timestamp
['campaign_id', 'click_id', 'click_time', 'creative_id', 'device_id', 'dummy_publisher_id', 'events.0', 'events.1', 'events.2', 'events.3', 'events.4', 'gaid', 'gross_adv_cost', 'idfa', 'install_count', 'ins

In [252]:
df.printSchema()

root
 |-- click_config: struct (nullable = true)
 |    |-- campaign_id: string (nullable = true)
 |    |-- click_id: string (nullable = true)
 |    |-- click_time: long (nullable = true)
 |    |-- creative_id: string (nullable = true)
 |    |-- device_id: string (nullable = true)
 |    |-- dummy_publisher_id: string (nullable = true)
 |    |-- events: struct (nullable = true)
 |    |    |-- 0: struct (nullable = true)
 |    |    |    |-- ev: string (nullable = true)
 |    |    |    |-- tm: long (nullable = true)
 |    |    |-- 1: struct (nullable = true)
 |    |    |    |-- ev: string (nullable = true)
 |    |    |    |-- pubEventId: long (nullable = true)
 |    |    |    |-- query: struct (nullable = true)
 |    |    |    |    |-- attribution_type: string (nullable = true)
 |    |    |    |-- sendToPub: long (nullable = true)
 |    |    |    |-- status: long (nullable = true)
 |    |    |    |-- ts: long (nullable = true)
 |    |    |-- 2: struct (nullable = true)
 |    |    |    |-- 

In [256]:
df = df.select(*a)

AnalysisException: [AMBIGUOUS_REFERENCE] Reference `campaign_id` is ambiguous, could be: [`campaign_id`, `campaign_id`].

In [257]:
df.printSchema()

root
 |-- campaign_id: string (nullable = true)
 |-- click_id: string (nullable = true)
 |-- click_time: long (nullable = true)
 |-- creative_id: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- dummy_publisher_id: string (nullable = true)
 |-- events: struct (nullable = true)
 |    |-- 0: struct (nullable = true)
 |    |    |-- ev: string (nullable = true)
 |    |    |-- tm: long (nullable = true)
 |    |-- 1: struct (nullable = true)
 |    |    |-- ev: string (nullable = true)
 |    |    |-- pubEventId: long (nullable = true)
 |    |    |-- query: struct (nullable = true)
 |    |    |    |-- attribution_type: string (nullable = true)
 |    |    |-- sendToPub: long (nullable = true)
 |    |    |-- status: long (nullable = true)
 |    |    |-- ts: long (nullable = true)
 |    |-- 2: struct (nullable = true)
 |    |    |-- ev: string (nullable = true)
 |    |    |-- pubEventId: long (nullable = true)
 |    |    |-- query: struct (nullable = true)
 |    |    |    |-